In [ ]:
from datetime import datetime

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, lit
from snowflake.core import Root

In [ ]:
# Initialize the Snowflake session
session = get_active_session()

# Create a Root instance
root = Root(session)

# Set the active database and schema
session.use_database('my_db')
session.use_schema('my_schema')

In [ ]:
execute notebook _01_REPORT_GEN__REPORT('a');
execute notebook _03_REPORT_GEN__SEND_EMAIL('a');

In [ ]:
execute notebook _01_REPORT_GEN__REPORT('b');
execute notebook _03_REPORT_GEN__SEND_EMAIL('b');

In [ ]:
execute notebook _01_REPORT_GEN__REPORT('c');
execute notebook _03_REPORT_GEN__SEND_EMAIL('c');

In [ ]:
# Convert the results of execution to DataFrames
a_df = _1_0_execute__a.to_df()
b_df = _1_1_execute__b.to_df()
c_df = _1_2_execute__c.to_df()

# Extract the 'status' column's first value from each DataFrame
a_result = a_df.select(col('"status"')).first()[0]
b_result = b_df.select(col('"status"')).first()[0]
c_result = c_df.select(col('"status"')).first()[0]

In [ ]:
# Get the current date
today = datetime.now()
current_date = today.strftime('%Y-%m-%d')

# Initialize warning variables
a_warning = 'Report A executed successfully'
b_warning = 'Report B executed successfully'
c_warning = 'Report C executed successfully'

# Check if each report executed successfully and update warnings if not
if a_result != 'Statement executed successfully.':
 a_warning = f'Report A did not execute properly on {current_date}'

if b_result != 'Statement executed successfully.':
    b_warning = f'Report B did not execute properly on {current_date}'

if c_result != 'Statement executed successfully.':
    c_warning = f'Report C did not execute properly on {current_date}'


In [ ]:
warnings_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Warnings</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            line-height: 1.5;
            margin: 20px;
        }}
    </style>
</head>
<body>
    <p>{a_warning}</p>
    <p>{b_warning}</p>
    <p>{c_warning}</p>
</body>
</html>
"""


send_email_sql = f"""
CALL SYSTEM$SEND_EMAIL(
    'report_email_integration',
    'me@email.com',
    'Reporting Failure',
    '{warnings_html}',
    'text/html'
)
"""

In [ ]:
if any(result != "Statement executed successfully." for result in (a_result, b_result, c_result)):
    session.sql(send_email_sql).collect()